# Movie Recommendation System

In this project, I will build a movie recommendation system based on a MovieLens archival dataset.

I will compare a system based on User to User Collaborative Filtering with Hamming Distance vs another system that uses Latent Factor Analysis for Collaborative Based Filtering and see if the movies recommended to a random active user are the same for both algorithms.



**About The Data Set:**
 
This data set consists of:
	- 100,000 ratings (1-5) from 943 users on 1682 movies. 
	- Each user has rated at least 20 movies. 
    - Simple demographic info for the users (age, gender, occupation, zip)

The data was collected through the MovieLens web site
(movielens.umn.edu) during the seven-month period from September 19th, 
1997 through April 22nd, 1998. This data has been cleaned up - users
who had less than 20 ratings or did not have complete demographic
information were removed from this data set. 


**DETAILED DESCRIPTIONS OF DATA FILES**

```
ml-data.tar.gz   -- Compressed tar file.  To rebuild the u data files do this:
                gunzip ml-data.tar.gz
                tar xvf ml-data.tar
                mku.sh

u.data     -- The full u data set, 100000 ratings by 943 users on 1682 items.
              Each user has rated at least 20 movies.  Users and items are
              numbered consecutively from 1.  The data is randomly
              ordered. This is a tab separated list of 
	         user id | item id | rating | timestamp. 
              The time stamps are unix seconds since 1/1/1970 UTC   

u.info     -- The number of users, items, and ratings in the u data set.

u.item     -- Information about the items (movies); this is a tab separated
              list of
              movie id | movie title | release date | video release date |
              IMDb URL | unknown | Action | Adventure | Animation |
              Children's | Comedy | Crime | Documentary | Drama | Fantasy |
              Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi |
              Thriller | War | Western |
              The last 19 fields are the genres, a 1 indicates the movie
              is of that genre, a 0 indicates it is not; movies can be in
              several genres at once.
              The movie ids are the ones used in the u.data data set.

u.genre    -- A list of the genres.

u.user     -- Demographic information about the users; this is a tab
              separated list of
              user id | age | gender | occupation | zip code
              The user ids are the ones used in the u.data data set.

u.occupation -- A list of the occupations.

u1.base    -- The data sets u1.base and u1.test through u5.base and u5.test
u1.test       are 80%/20% splits of the u data into training and test data.
u2.base       Each of u1, ..., u5 have disjoint test sets; this if for
u2.test       5 fold cross validation (where you repeat your experiment
u3.base       with each training and test set and average the results).
u3.test       These data sets can be generated from u.data by mku.sh.
u4.base
u4.test
u5.base
u5.test

ua.base    -- The data sets ua.base, ua.test, ub.base, and ub.test
ua.test       split the u data into a training set and a test set with
ub.base       exactly 10 ratings per user in the test set.  The sets
ub.test       ua.test and ub.test are disjoint.  These data sets can
              be generated from u.data by mku.sh.

allbut.pl  -- The script that generates training and test sets where
              all but n of a users ratings are in the training data.

mku.sh     -- A shell script to generate all the u data sets from u.data.
```



In [1]:
# Common Imports:
import pandas as pd
import numpy as np
import os

# allowing for any single variable to print out without using the print statement:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# import data:

dataFile='./data/ml-100k/u.data'
data = pd.read_csv(dataFile, sep="\t", header=0, names=["user_id","movie_id","rating", "timestamp"])

movieFile='./data/ml-100k/u.item'
movies = pd.read_csv(movieFile, sep="|", encoding='Latin-1', header=0, names=['movie_id', 'movie_title', 'release_date', 'video_release_date', 'IMDb_URL', 'Unknown', 'Action', 'Adventure', 'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'])

In [3]:
data.head()
movies.head()

,user_id,movie_id,rating,timestamp
0,186,302,3,891717742
1,22,377,1,878887116
2,244,51,2,880606923
3,166,346,1,886397596
4,298,474,4,884182806


,movie_id,movie_title,release_date,video_release_date,IMDb_URL,Unknown,Action,Adventure,Animation,Childrens,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,6,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,01-Jan-1995,NaN,http://us.imdb.com/Title?Yao+a+yao+yao+dao+wai...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
data.shape
movies.shape

(99999, 4)

(1681, 24)

In [5]:
# Data Cleaning - only retain values from movie-rating that actually have movie meta data
data = data[data["movie_id"].isin(movies.movie_id)]

In [6]:
data.shape

(99547, 4)

In [7]:
# Find the number of Movies each user has rated
MoviesPerUser = data.user_id.value_counts()
# Find the number of users who have rated a particular movie. 
UsersPerMovie = data.movie_id.value_counts()

MoviesPerUser
UsersPerMovie

405    737
655    684
13     635
450    539
276    517
416    492
537    489
303    483
234    479
393    447
181    434
279    433
429    413
846    405
7      403
94     399
682    398
308    396
92     387
293    387
222    386
201    385
59     381
435    378
378    374
880    367
417    364
896    361
592    359
758    357
      ... 
631     20
873     20
809     20
231     20
166     20
740     20
36      20
866     20
418     20
143     20
824     20
888     20
926     20
685     20
558     20
732     20
475     20
571     20
596     20
252     20
147     20
19      20
572     20
364     20
242     19
202     19
441     19
93      19
636     19
895     19
Name: user_id, Length: 943, dtype: int64

50      583
258     509
100     508
181     507
294     485
286     481
288     478
300     431
121     429
174     420
127     413
56      394
7       392
98      390
237     384
117     378
172     367
222     365
204     350
313     350
405     344
79      336
210     331
151     326
173     324
69      321
168     316
748     316
269     315
257     303
       ... 
1653      1
857       1
1587      1
1651      1
1624      1
1559      1
1460      1
599       1
1494      1
1366      1
1621      1
1557      1
1493      1
1461      1
1492      1
1655      1
1364      1
1236      1
852       1
1619      1
1363      1
1235      1
1526      1
1654      1
1682      1
1618      1
1681      1
1680      1
1616      1
1663      1
Name: movie_id, Length: 1681, dtype: int64

In [8]:
# More Data Cleaning
# Drop movies that dont have too many ratings
# lets only keep movies which have been rated by more than 10 users

data.shape
condition = data['movie_id'].isin(UsersPerMovie[UsersPerMovie > 10].index)
data = data[condition]
data.shape

(99547, 4)

(97170, 4)

#### These steps are to get the data out to CSV files for GOOGLE BIG QUERY:

In [10]:
data.head()
data.shape
movies.head()
movies.shape

,user_id,movie_id,rating,timestamp
0,186,302,3,891717742
1,22,377,1,878887116
2,244,51,2,880606923
3,166,346,1,886397596
4,298,474,4,884182806


(97170, 4)

,movie_id,movie_title,release_date,video_release_date,IMDb_URL,Unknown,Action,Adventure,Animation,Childrens,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,6,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,01-Jan-1995,NaN,http://us.imdb.com/Title?Yao+a+yao+yao+dao+wai...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


(1681, 24)

In [11]:
# convert data timestamp to datetime:
data['timestamp'] = pd.to_datetime(data['timestamp'], unit='s')

In [13]:
# convert movie release_date to datetime:
movies['release_date'] = pd.to_datetime(movies['release_date'])

In [22]:
# drop video_release_date:
movies = movies.drop(columns=['video_release_date'])

In [24]:
#### EXPORT TO CSV:
movies.to_csv(path_or_buf="data/movies.csv", index=False)
data.to_csv(path_or_buf="data/data.csv", index=False)

## User to User Collaborative Filtering with Hamming Distance for Recommendation Systems:

In [10]:
# Now lets create the user-item-rating matrix using a pivot table function:

userItemRatingMatrix=pd.pivot_table(data, values='rating',
                                    index=['user_id'], columns=['movie_id'])
userItemRatingMatrix.head()

movie_id,2,3,4,5,6,7,8,9,10,11,...,1421,1428,1444,1446,1451,1469,1478,1480,1483,1518
user_id,,,,,,,,,,,,,,,,,,,,,
1,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
userItemRatingMatrix.shape

(943, 1118)

Now, we have our user-item rating matrix. We have 943 users and 1,118 movies. Recall, we started off with 1,682 movies, but we whittled down the movie list to 1,118 by removing movies that had less than 10 reviews. 

Let's now write a function to compute k nearest neighbours of an arbitrary active user using hamming distance. This will be the K number of closest people who are similar to this active user; we can use their highest rated viewed movies to recommend to the "Active User. We will be using the hamming distance to find distances between users. 

In [12]:
from scipy.spatial.distance import hamming 

In [13]:
# To Find the k nearest neighbours of this active user first find the distance of active user to all other users
def nearestneighbours(user,K):
    # create a user df that contains all users except active user (we want to remove the active user so we don't recommend your own things to yourself)
    allUsers = pd.DataFrame(userItemRatingMatrix.index)
    allUsers = allUsers[allUsers.user_id != user]
    
    # Add a column to this df which contains distance of active user to each user
    allUsers["distance"] = allUsers["user_id"].apply(lambda x: hamming(userItemRatingMatrix.loc[user], userItemRatingMatrix.loc[x]))
    # print(allUsers.sort_values(["distance"],ascending=True).head()) # debug
    
    # sort the values of all users based on the distance metric, and will return the top K user ids - we want the closest number of users
    KnearestUsers = allUsers.sort_values(["distance"],ascending=True)["user_id"][:K]
    return KnearestUsers

In [14]:
# Lets test the function - Find the 5 nearest neighbours of the active user 523
print(nearestneighbours(523, 5))

710    711
150    151
449    450
415    416
17      18
Name: user_id, dtype: int64


**NOW WE KNOW WHAT THE NEAREST NEIGHBORS, WE WANT TO SEE WHAT THEY READ AND RECOMMEND THE TOP "N" MOVIES TO THAT USER**

In [15]:
def topN(user, userItemRatingMatrix, N=3):
    '''given a user_ID, it returns the top N number of movie recommendations by movie_id'''
    
    # find the 10 neighbors nearest to that user
    KnearestUsers = nearestneighbours(user,10)
    
    # get the ratings given by nearest neighbours. Note the userIemRatingMatrix index is the user_id
    NNRatings = userItemRatingMatrix[userItemRatingMatrix.index.isin(KnearestUsers)]
    
    # Find the average rating of each movie rated by nearest neighbours.
        # np.nanmean - Computes the arithmetic mean along the specified axis, ignoring NaNs.
        # then, we're dropping movies that don't have ratings / dropping na's
    avgRating = NNRatings.apply(np.nanmean).dropna()
    
    # drop the movies already seen by active user
    moviesAlreadyWatched = userItemRatingMatrix.loc[user].dropna().index
    avgRating = avgRating[~avgRating.index.isin(moviesAlreadyWatched)]
    topMovies = avgRating.sort_values(ascending=False).index[:N]
    return topMovies

**Let's test our final function "topN" for a random user and see the top recommendations:**

In [16]:
'Top 5 movies recommended to user 523 using User to User Collaborative Filtering with Hamming Distance'
for index in topN(523, userItemRatingMatrix, 5):
    print("Movie: ", movies.at[index, "movie_title"])

'Top 5 movies recommended to user 523 using User to User Collaborative Filtering with Hamming Distance'

Movie:  Tango Lesson, The (1997)
Movie:  Naked (1993)
Movie:  Once Were Warriors (1994)
Movie:  Supercop (1992)
Movie:  Caught (1996)


C:\Users\danie\Anaconda3\lib\site-packages\pandas\core\apply.py:242: RuntimeWarning: Mean of empty slice
  labels=labels)


# Collaborative Based Filtering using Latent Factor Analysis

There are many latent features in the user's rating matrix. There are many known and unknown things from a rating matrix for a specific user (R) - we can use a rating matrix and factorize that into two matrices (P and Q) and fill in the features using stochastic gradient descent. After that, we can then multiply those two matrices full of features together to get a predicted user-item rating matrix and recommend / predict the highest rated movies that the user has not seen.

In [17]:
# instead of storing the UserItemRating Matrix in a PIVOT TABLE, we are STORING IT AS A COO MATRIX in this instance:
    # It's a more efficient way to store the sparse matrix. Instead of having so many null values. Why not create a new matrix with just the values that are relevant and exactly what point they're at. It's a COO Matrix - stands for coordinate matrix.
    # The keys are the row, column index and the value is the rating!
from scipy.sparse import coo_matrix

In [18]:
# make a copy of our data before modifying it:
data_lfa = data.copy()

# need to change the user and books to categorical data.
data_lfa['user_id'] = data_lfa['user_id'].astype("category")
data_lfa['movie_id'] = data_lfa['movie_id'].astype("category")

In [19]:
# then, create the COO Matrix using those categories. It creates plot points on the original matrix for user_id and movie_id that point to the specific rating value. It is created in order to store the useritemrating matrix in smaller memory - to store a sparse matrix efficiently. 
    # rows are user_id
    # columnes are movie_id
    # it assigns categorical codes in ascending order to the user and movie IDs
R = coo_matrix((data_lfa['rating'].astype(float),
                       (data_lfa['user_id'].cat.codes,
                        data_lfa['movie_id'].cat.codes)))

**Exploring our COO Matrix**

In [20]:
# exploring our COO matrix at position 7:
R.shape
R.data[7] # our 7th rating
R.row[7] # user_id coordinate for this rating
R.col[7] # movie_id coordinate for this rating

(943, 1118)

3.0

304

428

In [21]:
# that 304 is the mapping of the user created by the .cat.code -- here we are printing out WHAT USER that belongs to:
data_lfa[data_lfa.user_id.cat.codes==304].head()

,user_id,movie_id,rating,timestamp
7,305,451,3,886324817
146,305,427,5,886323090
172,305,117,2,886324028
214,305,471,4,886323648
236,305,214,2,886323068


In [22]:
'What were all the ratings that user 305 gave?'
R.data[R.row == 304]

'What is the average rating that he gave for all his/her movies?'
np.nanmean(R.data[R.row == 304])

'How many movies did user 305 rate?'
len(R.data[R.row==304])

'What were all the ratings that user 305 gave?'

array([3., 5., 2., 4., 2., 4., 3., 4., 4., 5., 3., 1., 5., 5., 4., 4., 1.,
       5., 5., 4., 3., 4., 1., 1., 3., 5., 3., 1., 3., 5., 4., 2., 4., 3.,
       3., 5., 3., 2., 1., 3., 4., 4., 3., 3., 4., 2., 4., 3., 2., 5., 5.,
       2., 4., 1., 3., 4., 3., 4., 3., 4., 4., 2., 2., 4., 2., 3., 3., 1.,
       5., 3., 5., 3., 3., 3., 3., 4., 3., 4., 4., 3., 4., 5., 5., 3., 4.,
       4., 3., 3., 3., 4., 4., 5., 5., 2., 4., 4., 3., 4., 5., 4., 4., 3.,
       3., 4., 3., 2., 4., 3., 1., 3., 3., 4., 3., 5., 3., 4., 4., 3., 4.,
       4., 3., 4., 2., 4., 1., 4., 5., 4., 2., 3., 5., 4., 4., 3., 3., 3.,
       3., 2., 5., 2., 4., 5., 1., 3., 3., 4., 4., 3., 5., 3., 2., 2., 4.,
       4., 3., 3., 3., 5., 4., 4., 3., 3., 5., 5., 2., 3., 4., 3., 5., 4.,
       2., 5., 2., 2., 4., 4., 5., 4., 2., 3., 4., 3., 5., 2., 3., 2., 4.,
       4., 3., 3., 3., 3., 3., 5., 3., 2., 3., 2., 3., 3., 3., 4., 5., 3.,
       2., 5., 2., 3., 5., 5., 4., 3., 3., 3., 5.])

'What is the average rating that he gave for all his/her movies?'

3.4

'How many movies did user 305 rate?'

215

In [23]:
# this enumerate function will tell us the mappings for which category code belong with which user and which category codes belong with which movies:

dict(enumerate(data_lfa['movie_id'].cat.categories))
dict(enumerate(data_lfa['user_id'].cat.categories))

{0: 2,
 1: 3,
 2: 4,
 3: 5,
 4: 6,
 5: 7,
 6: 8,
 7: 9,
 8: 10,
 9: 11,
 10: 12,
 11: 13,
 12: 14,
 13: 15,
 14: 16,
 15: 17,
 16: 19,
 17: 20,
 18: 21,
 19: 22,
 20: 23,
 21: 24,
 22: 25,
 23: 26,
 24: 27,
 25: 28,
 26: 29,
 27: 30,
 28: 31,
 29: 32,
 30: 33,
 31: 35,
 32: 36,
 33: 38,
 34: 39,
 35: 40,
 36: 41,
 37: 42,
 38: 43,
 39: 44,
 40: 45,
 41: 46,
 42: 47,
 43: 48,
 44: 49,
 45: 50,
 46: 51,
 47: 52,
 48: 53,
 49: 54,
 50: 55,
 51: 56,
 52: 57,
 53: 58,
 54: 59,
 55: 60,
 56: 61,
 57: 62,
 58: 63,
 59: 64,
 60: 65,
 61: 66,
 62: 67,
 63: 68,
 64: 69,
 65: 70,
 66: 71,
 67: 72,
 68: 73,
 69: 76,
 70: 77,
 71: 78,
 72: 79,
 73: 80,
 74: 81,
 75: 82,
 76: 83,
 77: 84,
 78: 85,
 79: 86,
 80: 87,
 81: 88,
 82: 89,
 83: 90,
 84: 91,
 85: 92,
 86: 93,
 87: 94,
 88: 95,
 89: 96,
 90: 97,
 91: 98,
 92: 99,
 93: 100,
 94: 101,
 95: 102,
 96: 103,
 97: 105,
 98: 106,
 99: 107,
 100: 108,
 101: 109,
 102: 110,
 103: 111,
 104: 112,
 105: 114,
 106: 115,
 107: 116,
 108: 117,
 109: 118,
 

{0: 1,
 1: 2,
 2: 3,
 3: 4,
 4: 5,
 5: 6,
 6: 7,
 7: 8,
 8: 9,
 9: 10,
 10: 11,
 11: 12,
 12: 13,
 13: 14,
 14: 15,
 15: 16,
 16: 17,
 17: 18,
 18: 19,
 19: 20,
 20: 21,
 21: 22,
 22: 23,
 23: 24,
 24: 25,
 25: 26,
 26: 27,
 27: 28,
 28: 29,
 29: 30,
 30: 31,
 31: 32,
 32: 33,
 33: 34,
 34: 35,
 35: 36,
 36: 37,
 37: 38,
 38: 39,
 39: 40,
 40: 41,
 41: 42,
 42: 43,
 43: 44,
 44: 45,
 45: 46,
 46: 47,
 47: 48,
 48: 49,
 49: 50,
 50: 51,
 51: 52,
 52: 53,
 53: 54,
 54: 55,
 55: 56,
 56: 57,
 57: 58,
 58: 59,
 59: 60,
 60: 61,
 61: 62,
 62: 63,
 63: 64,
 64: 65,
 65: 66,
 66: 67,
 67: 68,
 68: 69,
 69: 70,
 70: 71,
 71: 72,
 72: 73,
 73: 74,
 74: 75,
 75: 76,
 76: 77,
 77: 78,
 78: 79,
 79: 80,
 80: 81,
 81: 82,
 82: 83,
 83: 84,
 84: 85,
 85: 86,
 86: 87,
 87: 88,
 88: 89,
 89: 90,
 90: 91,
 91: 92,
 92: 93,
 93: 94,
 94: 95,
 95: 96,
 96: 97,
 97: 98,
 98: 99,
 99: 100,
 100: 101,
 101: 102,
 102: 103,
 103: 104,
 104: 105,
 105: 106,
 106: 107,
 107: 108,
 108: 109,
 109: 110,
 110: 11

In [24]:
# this function will return the error between the actual rating and the ratings calculated from P and Q matrices.
    # it is a HELPER function that is calculating the ERROR as we run the Stochastic Gradient Descent. 
    # predicted rating is calculated using the formula: 
    # pow(rui-np.dot(P[u,:],Q[:,i]),2)+ lamda*(pow(norm(P[u,:]),2)+pow(norm(Q[:,i]),2

from numpy.linalg import norm
def error(R,P,Q,reg=0.02):
    '''inputs are R P Q which are the matrices and the output is the sum of the errors. The regularization parameter is the LABMDA in the error formula that we are trying to calculate. R is the original created COO Matrix'''
    ratings = R.data
    rows = R.row
    cols = R.col
    e = 0 
    # go through every rating (for user/item in the range)
    for ui in range(len(ratings)):
        # Save the rating, user code and movie code
        rui=ratings[ui]
        u = rows[ui]
        i = cols[ui]
        # if the rating exists (can only rate between 1 and 5), then calculate the error using the formula
        if rui>0:
            # Find the sum of errors using a dot product (np matrix multiplication)
            e= e + pow(rui-np.dot(P[u,:],Q[:,i]),2)+\
                reg*(pow(norm(P[u,:]),2)+pow(norm(Q[:,i]),2))
    return e

In [25]:
# Function that will calculate P and Q matrices using stochastic gradient descent
def SGD(R, K, reg=0.02, steps=10, lrate=0.001): 
    # Setup the dimensions using the shape of R
    # M - No. of users
    # N - No. of items
    # K - No. of features (F1, F2, F3, F4)
    # Intialise and Fille the P and Q Factor Matrices with random numbers
    M,N = R.shape
    P = np.random.rand(M,K)
    Q = np.random.rand(K,N)
    
    # calculate the initial RMSE with the help of our error function above:
    rmse = np.sqrt(error(R,P,Q,reg)/len(R.data))
    print("Initial RMSE: "+str(rmse))
    
    # complete the specified number of steps for gradient descent or if you reach a particular RMSE of less than 0.5. Feel free to change it here. Here it's ok with a rating error of 0.5. Remember RMSE is in the units of the problem. A rating error within 0.5 is acceptable.
    for step in range(steps):
        # for user and item for every rating
        for ui in range(len(R.data)):
            rui=R.data[ui] # rating for that user / item
            u = R.row[ui] # user
            i = R.col[ui] # item
            if rui>0: # if there is a rating. 
                # update P, Q in the direction of local minima
                eui=rui-np.dot(P[u,:],Q[:,i]) # error in that specific user/item = actual specific rating - the calculated (initially random) P and Q values located at that specific coordinate.
                # updating P and Q using partial derivatives - this is where the magic happens:
                P[u,:]=P[u,:]+lrate*2*(eui*Q[:,i]-reg*P[u,:])
                Q[:,i]=Q[:,i]+lrate*2*(eui*P[u,:]-reg*Q[:,i])
                
        # calculating a new RMSE after updating P and Q.
        rmse = np.sqrt(error(R,P,Q,reg)/len(R.data))
        if rmse<0.5:
            break
    print("Final RMSE: "+ str(rmse))
    return P,Q

In [26]:
# call the SGD to calculate P,Q based on the COO Matrix that we already have (R)
P, Q = SGD(R,K=2,lrate=0.0007,reg=0.01, steps=100)

Initial RMSE: 3.2422772500016097
Final RMSE: 0.9361918827629507


In [27]:
# creating our new rating matrix based off our calculated P and Q fully filled features:
R2 = np.dot(P, Q)

In [28]:
type(R2)
R2.shape
R2[0] # all the calculated ratings for this first user for every potential movie.
R2[:,0] # all the calculated ratings for this first movie for every potential user.

numpy.ndarray

(943, 1118)

array([3.39163576, 3.11629072, 3.70488491, ..., 2.52742516, 3.3427951 ,
       3.06061514])

array([3.39163576, 3.43449169, 2.82834125, 4.09777395, 2.96193322,
       3.06012608, 3.65490859, 3.50231706, 3.70813744, 3.61250917,
       3.20448481, 3.70452631, 3.08203233, 3.45216589, 2.8811322 ,
       3.83305255, 2.69304997, 3.35789001, 3.16997663, 2.89832074,
       2.97534664, 3.39393873, 3.18775831, 3.80313643, 3.39195052,
       2.87360599, 3.10838974, 3.4118204 , 3.38664964, 3.6073196 ,
       3.50150737, 3.13431373, 3.72408671, 4.09152712, 3.22440646,
       3.84062986, 3.45693181, 3.81577017, 3.42059043, 2.69467616,
       3.16243136, 3.29298416, 3.43835781, 3.2884969 , 3.59554183,
       3.65720642, 3.41600386, 3.25582897, 2.51054465, 3.17202703,
       2.95145355, 3.81236587, 3.43345744, 3.60516836, 3.16762889,
       3.44239203, 3.44042097, 3.29254947, 3.5187914 , 3.51392423,
       2.9745815 , 3.04418472, 2.91858993, 3.19213763, 3.39699876,
       3.24917777, 4.02079927, 2.95717864, 3.42812019, 3.203382  ,
       3.01563574, 3.32458319, 3.11838231, 3.29857101, 3.30162

In [29]:
# now, we can turn this into a dataframe using the previous userItemRatingMatrix index and columns to fill in the indices (user_ids) and columns (movie_ids)

pred_matrix = pd.DataFrame(data=R2,
              index=userItemRatingMatrix.index,    
              columns=userItemRatingMatrix.columns)  

In [30]:
pred_matrix.head()

movie_id,2,3,4,5,6,7,8,9,10,11,...,1421,1428,1444,1446,1451,1469,1478,1480,1483,1518
user_id,,,,,,,,,,,,,,,,,,,,,
1,3.391636,3.116291,3.704885,3.311908,3.918145,3.991341,4.138019,4.044701,4.023983,3.936414,...,3.557923,3.826610,2.655136,3.116571,4.002224,3.408106,2.961389,2.527425,3.342795,3.060615
2,3.434492,3.218912,3.692656,3.419828,3.941900,3.951309,4.154675,3.989783,3.995583,3.994103,...,3.593850,3.824780,2.696211,3.217256,4.114823,3.372073,3.041885,2.523368,3.453867,3.081437
3,2.828341,2.696663,2.998135,2.864169,3.227526,3.188356,3.395586,3.208758,3.232952,3.294950,...,2.953028,3.113366,2.225815,2.693892,3.433573,2.719591,2.536257,2.051925,2.894202,2.524655
4,4.097774,3.725933,4.512809,3.960527,4.749853,4.878366,5.021622,4.952473,4.910855,4.751052,...,4.304275,4.654389,3.203275,3.727472,4.797070,4.166661,3.551271,3.075927,3.996134,3.708891
5,2.961933,2.525323,3.418613,2.687389,3.501611,3.766233,3.724264,3.861093,3.759942,3.413038,...,3.135159,3.497434,2.295406,2.531584,3.302786,3.221634,2.452596,2.318846,2.705799,2.728215


In [31]:
def topR(user_id, user_matrix, pred_matrix, N=3):
    '''given a user_ID, sparse user-item-rating matrix, and a predicted matrix with all filled values, it returns the top N number of movie recommendations by movie_id'''
    
    # drop the movies already seen by active user
    moviesAlreadyWatched = user_matrix.loc[user_id].dropna().index
    # get movies for that user id, where the movie ids (columns) are NOT already watched, sort them.
    top_movies = pred_matrix.loc[user_id][~pred_matrix.columns.isin(moviesAlreadyWatched)].sort_values(ascending=False)
#     print('Top Movies with Rating:')
#     print(top_movies.head(N))
#     print()
    # return the top N movie IDs without the rating. Index is the rating.
    return top_movies.index[:N]

**Let's test out the top movies using Collaborative Based Filtering and Latent Factor Analysis and compare these results:***

In [32]:
'Top 5 movies recommended to user 523 using Collaborative Based Filtering with Latent Factor Analysis'
for index in topR(523, userItemRatingMatrix, pred_matrix, 5):
    print("Movie: ", movies.at[index, "movie_title"])

'Top 5 movies recommended to user 523 using Collaborative Based Filtering with Latent Factor Analysis'

Movie:  Paradise Lost: The Child Murders at Robin Hood Hills (1996)
Movie:  While You Were Sleeping (1995)
Movie:  My Fair Lady (1964)
Movie:  Aliens (1986)
Movie:  Meet Me in St. Louis (1944)


## Comparison of Recommendations:

If you recall, the movies recommended to use 523 using User to User Based Collaborative Filtering with Hemming Distance created these 5 movies:

```
Top 5 movies recommended to user 523 using User to User Collaborative Filtering with Hamming Distance

Movie:  Tango Lesson, The (1997)
Movie:  Naked (1993)
Movie:  Once Were Warriors (1994)
Movie:  Supercop (1992)
Movie:  Caught (1996)
```

These 5 movies are not the same. That might be due to the fact that we are looking at 1,118 movies. Let's see out of the top 100 movie recommendations, what percentage of recommendations overlap based on these two types of systems.

In [33]:
def compare_recommendations(user_id, userItemRatingMatrix, pred_matrix, N):
    latent_based = topR(user_id, userItemRatingMatrix, pred_matrix, N)
    user_based = topN(user_id, userItemRatingMatrix, N)
    overlap = np.sum(latent_based.isin(user_based))
    print(f'Overlap: {overlap} / {N}\nPercentage: {round(overlap/N * 100, 2)}%')

In [34]:
compare_recommendations(523, userItemRatingMatrix, pred_matrix, 100)

Overlap: 34 / 100
Percentage: 34.0%


35% accuracy between the two recommendation systems seems a bit low. Let's compare these two recommendation systems to 100 purely random movie choices and see how much higher this 35% overlap in recommendations would be in comparison.

In [35]:
import random

In [38]:
def compare_random(user_id, userItemRatingMatrix, pred_matrix, N):
    latent_based = topR(user_id, userItemRatingMatrix, pred_matrix, N)
    user_based = topN(user_id, userItemRatingMatrix, N)
    
    # setting a random seed for this to be stable across runs
    random.seed(7)
    random_recs = random.sample(set(userItemRatingMatrix.columns), N)

    overlap_lat = np.sum(latent_based.isin(random_recs))
    overlap_user = np.sum(user_based.isin(random_recs))
    
    print(f'Overlap of Latent Based with Random Choices: {overlap_lat} / {N}\nPercentage: {round(overlap_lat/N * 100, 2)}%')
    print(f'Overlap of User Based with Random Choices: {overlap_user} / {N}\nPercentage: {round(overlap_user/N * 100, 2)}%')

In [39]:
compare_random(523, userItemRatingMatrix, pred_matrix, 100)

Overlap of Latent Based with Random Choices: 13 / 100
Percentage: 13.0%
Overlap of User Based with Random Choices: 7 / 100
Percentage: 7.0%


C:\Users\danie\Anaconda3\lib\site-packages\pandas\core\apply.py:242: RuntimeWarning: Mean of empty slice
  labels=labels)


**So, at least there is a method to the madness of these recommendation systems. 35% overlap of recommendations with the systems vs 7-11% overlap of recommendations with random choices.**

Let's test it with a couple more users:

In [40]:
compare_recommendations(524, userItemRatingMatrix, pred_matrix, 100)
compare_random(524, userItemRatingMatrix, pred_matrix, 100)

Overlap: 30 / 100
Percentage: 30.0%
Overlap of Latent Based with Random Choices: 11 / 100
Percentage: 11.0%
Overlap of User Based with Random Choices: 3 / 100
Percentage: 3.0%


In [41]:
compare_recommendations(7, userItemRatingMatrix, pred_matrix, 100)
compare_random(7, userItemRatingMatrix, pred_matrix, 100)

Overlap: 45 / 100
Percentage: 45.0%
Overlap of Latent Based with Random Choices: 9 / 100
Percentage: 9.0%
Overlap of User Based with Random Choices: 4 / 100
Percentage: 4.0%
